In [7]:
import datetime
import hmac
import hashlib
import base64
import json
import pandas as pd
import time
import websocket
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import matplotlib.ticker as mticker
import sys
import math
from urllib.parse import urlencode
import datetime as dt
import logging
import requests
import os
from google.cloud import bigquery
import imp

In [8]:

sys.path.insert(0,"/home/eddie_akers_anchorlabs_com/trading/python")
from lib import talos_utils

sys.path.append(os.environ["HOME"] + "/anchorage/source/python/lib/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility # type: ignore
import utils as aql_utils # type: ignore

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.8f}'.format

api_key = 'ANC34AV1RAK2'
api_secret = 'mj9a70zpeve9rnam55kn0ql6c7sze3gw'

talos_wl_api = 'ANC34AV1RAK2'
talos_wl_secret_api = 'mj9a70zpeve9rnam55kn0ql6c7sze3gw'
utc_now = datetime.datetime.utcnow()
utc_datetime = utc_now.strftime("%Y-%m-%dT%H:%M:%S.000000Z")

talos_api = 'ANCHJEWQR3YH'
talos_secret = '3c4nnc7lg7mfg2ae2irz5kkueeei37pz'

host = "tal-42.prod.talostrading.com"
host_wl = "tal-160.prod.talostrading.com"

In [9]:
logging.getLogger().setLevel(logging.INFO)

sys.path.insert(0,"/home/eddie_akers_anchorlabs_com/anchorage/source/python/lending_recon/src")
import utils
sys.path.insert(0,"/home/eddie_akers_anchorlabs_com/anchorage/source/python/lending_recon/src/vault_utils.py")
import vault_utils

imp.reload(utils)
imp.reload(vault_utils)
api_key_str = "4dbd027304ea4b4dd4ea6944fb61ecc633fdd4e3ec5ba21b8d7f5347ff14805c"
signing_key_str = "0b3e32e78b4d13109bda7fe5ece8616c8116ebea4cc6a8f61b5276800979073d"

 ## Anchorage Vault Utility
avu = vault_utils.AnchorageVaultUtility(
    api_key_str,
    signing_key_str
)


In [10]:
talos = talos_utils.Talos(talos_api, talos_secret, host)
talos_wl = talos_utils.Talos(talos_wl_api, talos_wl_secret_api, host_wl)

In [11]:
def get_fireblocks_balances():

    from google.cloud import bigquery
    import pandas as pd

    client = bigquery.Client()

    query = """
    WITH LatestEntries AS (
    SELECT 
        ID,
        MAX(created_at) AS latest_created_at
    FROM 
        `production-191601.brokerage.fireblocks_vault_balances`
    GROUP BY 
        ID
    )

    SELECT 
    a.ID,
    a.created_at,
    a.available
    FROM 
    `production-191601.brokerage.fireblocks_vault_balances` a
    JOIN 
    LatestEntries b ON a.ID = b.ID AND a.created_at = b.latest_created_at;
    """

    query_job = client.query(query)
    df = query_job.result().to_dataframe()

    return df

def parse_asset(asset):
    # If the entry is already a dictionary, return it
    if isinstance(asset, dict):
        return asset
    # If it's a string, try to parse it as JSON
    elif isinstance(asset, str):
        try:
            return json.loads(asset)
        except json.JSONDecodeError:
            return None  # Handle invalid JSON strings
    else:
        return None  # Handle unexpected types

    
def get_aplo_balances():
    sys.path.insert(0,os.environ["HOME"]+"/trading/python/")
    from lib import aplo
    aplo_key = "-xCvTl8CYBvFwD8MnMQsxdp4lItrun40"
    headers = {'X-API-Key': aplo_key}
    aplo_client = aplo.AploClient(aplo_key, debug=False)
    aplo_balances = pd.DataFrame(aplo_client.get_balances())   
    
    return aplo_balances

def get_aplo_open_orders():
    sys.path.insert(0,os.environ["HOME"]+"/trading/python/")
    from lib import aplo
    aplo_key = "-xCvTl8CYBvFwD8MnMQsxdp4lItrun40"
    headers = {'X-API-Key': aplo_key}
    aplo_client = aplo.AploClient(aplo_key, debug=False)
    aplo_orders = pd.json_normalize(aplo_client.get_orders())   

    
    return aplo_orders

def get_ads_balances():
    import pandas as pd
    from io import StringIO
    import imp
    from oauth2client.service_account import ServiceAccountCredentials
    import matplotlib.pyplot as plt
    from IPython.core.display import display, HTML

    import requests
    import json
    import os
    import time
    from datetime import datetime
    import logging
    
    data = avu.query_all_vaults()

    flattened_data = []
    for account in data:
        account_name = account['accountName']
        for asset in account['assets']:
            flattened_data.append({
                'accountName': account_name,
                'assetType': asset['assetType'],
                'availableBalance_currentPrice': asset['availableBalance']['currentPrice'],
                'availableBalance_currentUSDValue': asset['availableBalance']['currentUSDValue'],
                'availableBalance_quantity': asset['availableBalance']['quantity'],
                'totalBalance_currentPrice': asset['totalBalance']['currentPrice'],
                'totalBalance_currentUSDValue': asset['totalBalance']['currentUSDValue'],
                'totalBalance_quantity': asset['totalBalance']['quantity'],
                'vaultId': asset['vaultId'],
                'vaultName': asset['vaultName'],
                'walletId': asset['walletId']
            })

    # Creating the DataFrame
    df = pd.DataFrame(flattened_data)

    return df

def get_open_orders_wl():
    talos_counterparties_wl = talos_wl.get_orders(statuses="New,PartiallyFilled")

    talos_counterparties_wl = pd.DataFrame(talos_counterparties_wl)
    df = talos_counterparties_wl .sort_values(by='Timestamp', ascending=False).reset_index(drop=True)
    df = df[df['User'] != "Anchorage Trader"]
    df = df[df['SubAccount'] != "Hold"]
    parameters_expanded = df['Parameters'].apply(pd.Series)
    df_expanded = pd.concat([df.drop(columns=['Parameters']), parameters_expanded], axis=1)

    columns = [ "Timestamp", "User", "Symbol", "OrderID", "ClOrdID", "SubmitTime", "ExecID", 
    "Side", "TransactTime", "OrdStatus", "OrderQty", "OrdType", "Price", "Currency", 
    "LeavesQty", "CumQty", "AvgPx", "TimeInForce", "CumAmt", "CumFee", "DecisionStatus", 
    "AmountCurrency", "SessionID", "ExpectedFillPrice", "ExpectedFillQty", "Strategy", 
    "LastRequestTimestamp", "Revision", "Markets", "SubAccount", 
    "RequestSource", "LastRequestSource", "FeeCurrency", "AvgPxAllIn", "EndTime", 
    "LastTradeTime", "TransactedAssets","StartTime"]
    df = df_expanded[columns]
    

# Combine the original DataFrame with the expanded DataFrame

    return df

def get_open_orders():
    talos_counterparties = talos.get_orders(statuses="New,PartiallyFilled")

    talos_counterparties = pd.DataFrame(talos_counterparties)
    df = talos_counterparties.sort_values(by='Timestamp', ascending=False).reset_index(drop=True)
    columns = [ "Timestamp", "User", "Symbol", "OrderID", "ClOrdID", "SubmitTime", "ExecID", 
    "Side", "TransactTime", "OrdStatus", "OrderQty", "OrdType", "Price", "Currency", 
    "LeavesQty", "CumQty", "AvgPx", "TimeInForce", "CumAmt", "CumFee", "DecisionStatus", 
    "AmountCurrency", "SessionID", "ExpectedFillPrice", "ExpectedFillQty", "Strategy", 
    "LastRequestTimestamp", "Revision", "Markets", "Parameters", "SubAccount", 
    "RequestSource", "LastRequestSource", "FeeCurrency", "AvgPxAllIn", "EndTime", 
    "LastTradeTime", "TransactedAssets"]
    df = df[columns]
    return df
from pandas import json_normalize
def get_market_exposure():
    df = talos_wl.get_market_exposure()
    df = pd.DataFrame(df)
    unfurled_df = json_normalize(df['data'].explode())
    final_df = df.join(unfurled_df)
    #final_df = finaldf.drop(columns='data')
    return final_df

In [12]:
import time

while True:
    # Your existing code
    markets = "kraken,okex,coinbase,wintermute,cumberland,janestreet,galaxy,b2c2,bybit"
    talos_balances_wl = talos_wl.get_balances(markets=markets)
    talos_balances_wl = pd.DataFrame(talos_balances_wl)
    columns_to_drop = ['Status','MarketAccountID','ReconStatus']
    talos_balances_wl = talos_balances_wl.drop(columns=columns_to_drop)
    talos_balances_wl['AvailableAmount'] = pd.to_numeric(talos_balances_wl['AvailableAmount'], errors='coerce')

    coins = talos_balances_wl.Currency.unique()
    #cmc_map_df = cmc_map(coins)
    #cmc_prices_df = cmc_prices(cmc_map_df['Crypto'].tolist())
    #cmc_prices_df = cmc_prices_df.rename(columns={'Crypto': 'Currency'})

    #merged_df = pd.merge(talos_balances_wl, cmc_prices_df, left_on='Currency', right_on='Currency', how='left')
    #merged_df.loc[merged_df['Currency'] == 'USD', 'Price'] = 1
    #merged_df['MarketValue'] = merged_df['Price'] * merged_df['AvailableAmount']

    # Load params
    os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
    google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
    gsheet_key = json.loads(google_sheet_key)
    worksheet_name = "A1 Risk Dashboard"
    tab_name = "talosData"

    gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
    gsu.dump_current_sheet(tab_name, talos_balances_wl)

    tab_name = "fireblocks_balances"

    fireblocks_balances = get_fireblocks_balances()
    gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
    gsu.dump_current_sheet(tab_name, fireblocks_balances)

    tab_name = "aplo_balances"

    aplo_balances = get_aplo_balances()
    gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
    gsu.dump_current_sheet(tab_name, aplo_balances)

    tab_name = "ads_balances"

    ads_balances = get_ads_balances()
    gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
    gsu.dump_current_sheet(tab_name, ads_balances)

    tab_name = "Open_Orders"

    #open_orders_wl = get_open_orders_wl()
    open_orders = get_open_orders()
    #merged_orders = pd.merge(open_orders_wl, open_orders, on='order_id', how='inner')

    gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
    gsu.dump_current_sheet(tab_name, open_orders)

    tab_name = "Open_Orders_wl"

    #open_orders_wl = get_open_orders_wl()
    open_orders = get_open_orders_wl()
    #merged_orders = pd.merge(open_orders_wl, open_orders, on='order_id', how='inner')

    gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
    gsu.dump_current_sheet(tab_name, open_orders)

    tab_name = "aplo_orders"

    #open_orders_wl = get_open_orders_wl()
    open_orders = get_aplo_open_orders()
    #merged_orders = pd.merge(open_orders_wl, open_orders, on='order_id', how='inner')

    gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
    gsu.dump_current_sheet(tab_name, open_orders)

    tab_name = "market_exposure"

    market_exposure = get_market_exposure()
   

    gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
    gsu.dump_current_sheet(tab_name, market_exposure)

    tab_name = "completed_orders"

    df = talos_wl.get_orders(start_date="2024-10-10T00:00:00.000000Z",statuses="DoneForDay")
    
    df= pd.DataFrame(df)
    df = df[df['SubAccount'] != "Hold"]
    df = df[df['User'] != "Anchorage Trader"]
    df = df[df['RequestSource'] != "Customer"]
    gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
    gsu.dump_current_sheet(tab_name, df)

    # Sleep for 10 seconds before running the loop again
    time.sleep(60)


[2024-11-26T13:35:09.398+0000] {vault_utils.py:36} INFO - getting vaults path: https://api.anchorage.com/v2/vaults?limit=100
[2024-11-26T13:35:16.335+0000] {aplo.py:96} INFO - Fetching page 0
[2024-11-26T13:35:16.466+0000] {aplo.py:96} INFO - Fetching page 1
[2024-11-26T13:35:16.596+0000] {aplo.py:96} INFO - Fetching page 2
[2024-11-26T13:35:16.714+0000] {aplo.py:96} INFO - Fetching page 3
[2024-11-26T13:35:16.823+0000] {aplo.py:96} INFO - Fetching page 4
[2024-11-26T13:35:16.823+0000] {aplo.py:109} INFO - Finished pagination. Total records fetched: 185
[2024-11-26T13:36:40.578+0000] {vault_utils.py:36} INFO - getting vaults path: https://api.anchorage.com/v2/vaults?limit=100
[2024-11-26T13:36:47.126+0000] {aplo.py:96} INFO - Fetching page 0
[2024-11-26T13:36:47.249+0000] {aplo.py:96} INFO - Fetching page 1
[2024-11-26T13:36:47.371+0000] {aplo.py:96} INFO - Fetching page 2
[2024-11-26T13:36:47.490+0000] {aplo.py:96} INFO - Fetching page 3
[2024-11-26T13:36:47.601+0000] {aplo.py:96} INF

KeyboardInterrupt: 